In [17]:
import pandas as pd
from ast import literal_eval
import torch

In [4]:
behaviors_parsed = pd.read_table('../../data/train/behaviors_parsed.tsv')

In [5]:
behaviors_parsed.head()

,user,clicked_news,candidate_news,clicked
0,1,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N94157 N58592 N58258,1 0 0
1,1,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78699 N39404 N7578,1 0 0
2,1,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N71090 N97778 N32954,1 0 0
3,1,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N31174 N27822 N26368,1 0 0
4,2,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N25587 N125161 N62024,1 0 0


In [27]:
news_parsed = pd.read_table(
            '../../data/train/news_parsed.tsv',
            index_col='id',
            usecols=['id'] + ['title'],
            converters={
                attribute: literal_eval
                for attribute in set(['title']) & set([
                    'title', 'abstract', 'title_entities', 'abstract_entities',
                    'title_roberta', 'title_mask_roberta', 'abstract_roberta',
                    'abstract_mask_roberta'
                ])
            })

In [28]:
news_parsed.head()

,title
id,
N88753,"[1, 2, 3, 4, 5, 6, 7, 5, 8, 6, 9, 10, 11, 0, 0..."
N45436,"[23, 24, 25, 26, 27, 28, 0, 0, 0, 0, 0, 0, 0, ..."
N23144,"[40, 41, 42, 43, 44, 45, 0, 0, 0, 0, 0, 0, 0, ..."
N86255,"[58, 59, 56, 60, 61, 62, 1, 63, 30, 64, 52, 65..."
N93187,"[1, 66, 59, 67, 30, 68, 69, 70, 1, 71, 59, 72,..."


In [41]:
news_id2int = {x: i for i, x in enumerate(news_parsed.index)}
news2dict = news_parsed.to_dict('index')

In [42]:
for key1 in news2dict.keys():
            for key2 in news2dict[key1].keys():
                news2dict[key1][key2] = torch.tensor(
                    news2dict[key1][key2])

In [31]:
padding_all = {
            'category': 0,
            'subcategory': 0,
            'title': [0] * 20,
            'abstract': [0] * 50,
            'title_entities': [0] * 20,
            'abstract_entities': [0] * 50,
            'title_roberta': [0] * 20,
            'title_mask_roberta': [0] * 20,
            'abstract_roberta': [0] * 50,
            'abstract_mask_roberta': [0] * 50
        }
for key in padding_all.keys():
    padding_all[key] = torch.tensor(padding_all[key])

In [33]:
padding_all

{'category': tensor(0),
 'subcategory': tensor(0),
 'title': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'abstract': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'title_entities': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'abstract_entities': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'title_roberta': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'title_mask_roberta': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'abstract_roberta': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'abstrac

In [35]:
padding = {
    k: v
    for k, v in padding_all.items()
    if k in ['title']
}

In [36]:
padding

{'title': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [44]:
def getitem(idx):
        num_clicked_news_a_user = 50
        item = {}
        row = behaviors_parsed.iloc[idx]
        item["clicked"] = list(map(int, row.clicked.split()))
        item["candidate_news"] = [
            news2dict[x] for x in row.candidate_news.split()
        ]
        item["clicked_news"] = [
            news2dict[x]
            for x in row.clicked_news.split()[:num_clicked_news_a_user]
        ]
        repeated_times = num_clicked_news_a_user - \
            len(item["clicked_news"])
        
        item["clicked_news"] = [padding] * repeated_times + item["clicked_news"]

        return item

In [45]:
item = getitem(0)

In [48]:
item

{'clicked': [1, 0, 0],
 'candidate_news': [{'title': tensor([  559, 55671,   686,  1577,  1469,    59,  3237,  1715,     5,  2192,
               5,    70,  2582,  4232,   214,   392,    30, 77128,   109,   935])},
  {'title': tensor([  270,  4405, 12408,   746,    43,   258,   770,     8,  2213,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0])},
  {'title': tensor([ 1954,   854,  1093,   162,  2008,   345,    51,  1807,    90,  1927,
           10938,    22,     0,     0,     0,     0,     0,     0,     0,     0])}],
 'clicked_news': [{'title': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
  {'title': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
  {'title': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
  {'title': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
  {'title': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
  {'t